# Introduction

私の初めての機械学習です。このnotebookはpythonで書かれています。

In [10]:
%matplotlib inline
import numpy as np
import pandas as pd
import re as re

# pandasを使ってcsvファイルの読み込み
train = pd.read_csv("train.csv", header = 0, dtype = {'Age': np.float64})
test = pd.read_csv("test.csv", header = 0, dtype = {'Age': np.float64})
full_data = [train, test]

print (train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


# Feature Engineering

## 1. Pclass (Passenger class)

In [19]:
# groupby()でPclassによってグループ化し、Survivedの算術平均を求める
print (train[["Pclass", "Survived"]].groupby(["Pclass"], as_index = False).mean())

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


mean()によってPclassごとにSurvivedした算術平均を求める。
ここでは、生き延びた人を1、亡くなった人を0と分類しているため、1stクラスに宿泊していた人たちの6割以上が生き延びたことがわかる。

## 2. Sex

In [18]:
print (train[["Sex", "Survived"]].groupby(["Sex"], as_index = False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


## 3. SibSp and Parch
siblings/spouseとchildren/parentsの数量を合算し、新しくFamilySizeという項目をつくる。

In [22]:
# for文内でSibSpとParchを合算し、FamilySizeに代入
for dataset in full_data:
    dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1
print(train[["FamilySize", "Survived"]].groupby(["FamilySize"], as_index = False).mean())

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000


相関がありそうなので、1人(Alone)とそれ以外に分けます。1ならAlone、0ならNOT Aloneです。

In [26]:
for dataset in full_data:
    dataset["Alone"] = 0 # 変数の初期化
    dataset.loc[dataset["FamilySize"] == 1, "Alone"] = 1 # dataset["FamilySize"] == 1である行
                                                        #の"Alone"列に1を代入
print(train[["Alone", "Survived"]].groupby(["Alone"], as_index = False).mean())

   Alone  Survived
0      0  0.505650
1      1  0.303538
